In [10]:
import win32gui,time
import win32process
import time, win32con, win32api, win32gui, ctypes
import time, win32con, win32api, win32gui, ctypes
import win32clipboard
import pandas as pd

#현재 실행 중인 모든 창 핸들 가져오기
def kakao_room_serach():
    #1.현재 실행 중인 모든 창 핸들 가져오기
    while True:
        windows = []
        win32gui.EnumWindows(lambda hwnd, lParam: windows.append(hwnd), None)
        kakao_title=[]
        #1.카카오톡 프로세스 저장
        for window in windows:
            try:
                # 창 제목, 윈도우 클래스 이름, 해당 프로세스의 PID 출력
                window_title = win32gui.GetWindowText(window)
                window_class_name = win32gui.GetClassName(window)
                _, process_id = win32process.GetWindowThreadProcessId(window)
                # print(f"창 제목: {window_title}, 윈도우 클래스 이름: {window_class_name}, PID: {process_id}")
                
                if window_title == "카카오톡":
                    카카오톡프로세스아이디=process_id
                    print(f"카카오톡 프로세스 아이디:{카카오톡프로세스아이디}")
            except:
                pass
        #2.카카오톡 대화창 찾기.
        for window in windows:
            try:
                # 창 제목, 윈도우 클래스 이름, 해당 프로세스의 PID 출력
                window_title = win32gui.GetWindowText(window)
                window_class_name = win32gui.GetClassName(window)
                _, process_id = win32process.GetWindowThreadProcessId(window)
                # print(f"창 제목: {window_title}, 윈도우 클래스 이름: {window_class_name}, PID: {process_id}")
                #카카오톡 대화창은 클래스명이 #32770이다. 
                if process_id==카카오톡프로세스아이디 and window_class_name == "#32770":
                    kakao_title.append(window_title)  
            except:
                pass
        if len(kakao_title) < 1:
            print(f"카카오톡 대화창이 없습니다.")
            time.sleep(5)
            win32api.MessageBox(win32con.NULL, '카카오톡 창이 업습니다.', '카카오톡 창 과다.', win32con.MB_OK)
        elif len(kakao_title) > 1:
            print(f"한개 이상의 창이 켜져 있습니다.")
            win32api.MessageBox(win32con.NULL, '카카오톡 창이 한개 이상켜져있습니다.', '카카오톡 창 과다.', win32con.MB_OK)
            time.sleep(5)
        elif len(kakao_title) == 1:
            print(f"대화창수집완료하였습니다:{kakao_title[0]} ")
            return kakao_title[0]


# 조합키 쓰기 위해
def PostKeyEx(hwnd, key, shift, specialkey):
    global w
    w = win32con
    #사전실행 변수들.
    PBYTE256 = ctypes.c_ubyte * 256
    _user32 = ctypes.WinDLL("user32")
    GetKeyboardState = _user32.GetKeyboardState
    SetKeyboardState = _user32.SetKeyboardState
    PostMessage = win32api.PostMessage
    SendMessage = win32gui.SendMessage
    FindWindow = win32gui.FindWindow
    IsWindow = win32gui.IsWindow
    GetCurrentThreadId = win32api.GetCurrentThreadId
    GetWindowThreadProcessId = _user32.GetWindowThreadProcessId
    AttachThreadInput = _user32.AttachThreadInput
    MapVirtualKeyA = _user32.MapVirtualKeyA
    MapVirtualKeyW = _user32.MapVirtualKeyW
    MakeLong = win32api.MAKELONG
    #실질적으로 작동하는 함수들이다.
    if IsWindow(hwnd):

        ThreadId = GetWindowThreadProcessId(hwnd, None)

        lparam = MakeLong(0, MapVirtualKeyA(key, 0))
        msg_down = w.WM_KEYDOWN
        msg_up = w.WM_KEYUP

        if specialkey:
            lparam = lparam | 0x1000000

        if len(shift) > 0:  # Если есть модификаторы - используем PostMessage и AttachThreadInput
            pKeyBuffers = PBYTE256()
            pKeyBuffers_old = PBYTE256()

            SendMessage(hwnd, w.WM_ACTIVATE, w.WA_ACTIVE, 0)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, True)
            GetKeyboardState(ctypes.byref(pKeyBuffers_old))

            for modkey in shift:
                if modkey == w.VK_MENU:
                    lparam = lparam | 0x20000000
                    msg_down = w.WM_SYSKEYDOWN
                    msg_up = w.WM_SYSKEYUP
                pKeyBuffers[modkey] |= 128

            SetKeyboardState(ctypes.byref(pKeyBuffers))
            time.sleep(0.01)
            PostMessage(hwnd, msg_down, key, lparam)
            time.sleep(0.01)
            PostMessage(hwnd, msg_up, key, lparam | 0xC0000000)
            time.sleep(0.01)
            SetKeyboardState(ctypes.byref(pKeyBuffers_old))
            time.sleep(0.01)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, False)
            
        else:  # Если нету модификаторов - используем SendMessage
            SendMessage(hwnd, msg_down, key, lparam)
            SendMessage(hwnd, msg_up, key, lparam | 0xC0000000)


# PostKeyEx(hwndListControl, ord('A'), [w.VK_CONTROL], False)
# PostKeyEx(hwndListControl, ord('C'), [w.VK_CONTROL], False)



##챗팅방 찾아가기###
def chatroom_go(kakao_title):
    hwndkakao = win32gui.FindWindow(None, "카카오톡")
    hwndkakao_edit1 = win32gui.FindWindowEx( hwndkakao, None, "EVA_ChildWindow", None)
    hwndkakao_edit2_1 = win32gui.FindWindowEx( hwndkakao_edit1, None, "EVA_Window", None)
    hwndkakao_edit2_2 = win32gui.FindWindowEx( hwndkakao_edit1, hwndkakao_edit2_1, "EVA_Window", None)
    hwndkakao_edit3 = win32gui.FindWindowEx( hwndkakao_edit2_2, None, "Edit", None)

    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0,kakao_title)  
    #안정성위해필요
    time.sleep(1)   
    enter(hwndkakao_edit3)
    #글자 초기화 시켜놓기
    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0, "")

def enter(hwnd):
    win32api.PostMessage(hwnd, win32con.WM_KEYDOWN, win32con.VK_RETURN, 0)
    time.sleep(0.01)
    win32api.PostMessage(hwnd, win32con.WM_KEYUP, win32con.VK_RETURN, 0)
    time.sleep(0.5)

def send_msg(kakao_title:str,msg:str):
    ###타겟 챗팅방###
    hwndMain = win32gui.FindWindow(None,kakao_title)
    hwndEdit = win32gui.FindWindowEx( hwndMain, None, "RICHEDIT50W", None)
    # win32gui.SetForegroundWindow(hwndMain)

    ###타겟 챗팅방에서 메세지 전송###
    win32api.SendMessage(hwndEdit,win32con.WM_SETTEXT, 0, msg) # 채팅창 입력
    enter(hwndEdit)

def send_img(kakao_title,imgpath):
    ###타겟 챗팅방###
    hwndMain = win32gui.FindWindow( None,kakao_title)
    hwndEdit = win32gui.FindWindowEx( hwndMain, None, "RICHEDIT50W", None)
    # win32gui.SetForegroundWindow(hwndMain)

    ###타겟 챗팅방에서 메세지 전송###
    win32api.SendMessage(hwndEdit,win32con.WM_SETTEXT, 0, "진혁아 안녕?") # 채팅창 입력
    enter(hwndEdit)

    ###타겟 챗팅방에서 이미지 보내기###
    cliboard_imacopy(imgpath)

    time.sleep(0.5)
    win32gui.PostMessage(hwndEdit, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON,0) #클릭을 안하면 복붙이 씹힌다.
    PostKeyEx(hwndEdit, ord('V'), [w.VK_CONTROL], False)
    PostKeyEx(hwndEdit, ord('V'), [w.VK_CONTROL], False)

    time.sleep(0.5)
    hwndMain2 = win32gui.FindWindow( None,"#32770 (Dialog)")
    hwndEdit3 = win32gui.FindWindowEx(hwndMain2, None, "#32770", None)
    enter(hwndEdit3)

    win32gui.PostMessage(hwndMain, win32con.WM_CLOSE, 0, 0)


#이미지 클립보드에 복사하기.
def cliboard_imacopy(filepath):
    from io import BytesIO
    import win32clipboard
    from PIL import Image
    def send_to_clipboard(clip_type, data):
        win32clipboard.OpenClipboard()
        win32clipboard.EmptyClipboard()
        win32clipboard.SetClipboardData(clip_type, data)
        win32clipboard.CloseClipboard()

    # filepath ='5.png'
    image = Image.open(filepath)
    output = BytesIO()
    image.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]
    output.close()
    send_to_clipboard(win32clipboard.CF_DIB, data) #클립보드 복사.

#총대장함수이다. 이것만 실행하면 된다.
def kakao_work(kakao_title,imaga_filepath):
    chatroom_go(kakao_title)
    send_msg(kakao_title,imaga_filepath)


def kakakao_getText(hwnd):
    w = win32con
    hwndMain = win32gui.FindWindow(None,hwnd)
    hwndEdit = win32gui.FindWindowEx(hwndMain, None, "EVA_VH_ListControl_Dblclk", None)
    PostKeyEx(hwndEdit, ord('A'), [w.VK_CONTROL], False)
    time.sleep(0.5)
    PostKeyEx(hwndEdit, ord('C'), [w.VK_CONTROL], False)
    win32clipboard.OpenClipboard()
    content = win32clipboard.GetClipboardData()
    win32clipboard.CloseClipboard()
    print(content)
    a = content.split('\r\n')   # \r\n 으로 스플릿 __ 대화내용 인용의 경우 \r 때문에 해당안됨
    df = pd.DataFrame(a)    # DF 으로 바꾸기
    print(df)
    df[0] = df[0].str.replace('\[([\S\s]+)\] \[(오전|오후)([0-9:\s]+)\] ', '')  # 정규식으로 채팅내용만 남기기
    print(df[0])
    print(df.index[-2], df.iloc[-2, 0])
    return df.index[-2], df.iloc[-2, 0]
    
def kakao_getinputText(hwnd):
    hwndMain = win32gui.FindWindow(None,hwnd)
    hwndEdit = win32gui.FindWindowEx(hwndMain, None, "RICHEDIT50W", None)
    PostKeyEx(hwndEdit, ord('A'), [win32con.VK_CONTROL], False)
    time.sleep(0.5)
    PostKeyEx(hwndEdit, ord('C'), [win32con.VK_CONTROL], False)
    win32clipboard.OpenClipboard()
    content = win32clipboard.GetClipboardData()
    win32clipboard.CloseClipboard()
    print(content)
    return content


# chatroom_go("상준")
# PostKeyEx(hwndListControl, ord('A'), [w.VK_CONTROL], False)
# PostKeyEx(hwndListControl, ord('C'), [w.VK_CONTROL], False)
# win32gui.GetWindowText(edit_control)


#------사용법------#
if __name__=="__main__": 
    #1.퍈매자 채팅방 이름 찾기.
    # kakao_room_serach()

    #2.채팅방에 이미지 보내기.
    # img_path='C:/Users/lsh92/Desktop/asdasd.png'
    # send_img("상준",img_path)

    #3.채팅방에 메세지 보내기.
    chatroom_go("상준")
    send_msg("상준","어찌 잘 지내고 있니?")
    # kakakao_getText("상준")
    # kakao_getinputText("상준")

In [26]:
import time, win32con, win32api, win32gui, ctypes
import requests
from bs4 import BeautifulSoup
# from apscheduler.schedulers.background import BackgroundScheduler
from pywinauto import clipboard # 채팅창내용 가져오기 위해
# import pandas as pd # 가져온 채팅내용 DF로 쓸거라서


# # 카톡창 이름, (활성화 상태의 열려있는 창)
kakao_opentalk_name = '메모장'
chat_command = '실검 알려줘'  # 테스트용..

PBYTE256 = ctypes.c_ubyte * 256
_user32 = ctypes.WinDLL("user32")
GetKeyboardState = _user32.GetKeyboardState
SetKeyboardState = _user32.SetKeyboardState
PostMessage = win32api.PostMessage
SendMessage = win32gui.SendMessage
FindWindow = win32gui.FindWindow
IsWindow = win32gui.IsWindow
GetCurrentThreadId = win32api.GetCurrentThreadId
GetWindowThreadProcessId = _user32.GetWindowThreadProcessId
AttachThreadInput = _user32.AttachThreadInput

MapVirtualKeyA = _user32.MapVirtualKeyA
MapVirtualKeyW = _user32.MapVirtualKeyW

MakeLong = win32api.MAKELONG
w = win32con


# # 채팅방에 메시지 전송
def kakao_sendtext(chatroom_name, text):
    # # 핸들 _ 채팅방
    hwndMain = win32gui.FindWindow( None, chatroom_name)
    hwndEdit = win32gui.FindWindowEx( hwndMain, None, "RichEdit20W", None)

    win32api.SendMessage(hwndEdit, win32con.WM_SETTEXT, 0, text)
    SendReturn(hwndEdit)

# # 채팅내용 가져오기
def copy_chatroom(chatroom_name):
    # # 핸들 _ 채팅방
    hwndMain = win32gui.FindWindow( None, chatroom_name)
    hwndListControl = win32gui.FindWindowEx(hwndMain, None, "EVA_VH_ListControl_Dblclk", None)

    # #조합키, 본문을 클립보드에 복사 ( ctl + c , v )
    PostKeyEx(hwndListControl, ord('A'), [w.VK_CONTROL], False)
    time.sleep(1)
    PostKeyEx(hwndListControl, ord('C'), [w.VK_CONTROL], False)
    ctext = clipboard.GetData()
    print(ctext)
    return ctext


# 조합키 쓰기 위해
def PostKeyEx(hwnd, key, shift, specialkey):
    if IsWindow(hwnd):

        ThreadId = GetWindowThreadProcessId(hwnd, None)

        lparam = MakeLong(0, MapVirtualKeyA(key, 0))
        msg_down = w.WM_KEYDOWN
        msg_up = w.WM_KEYUP

        if specialkey:
            lparam = lparam | 0x1000000

        if len(shift) > 0:
            pKeyBuffers = PBYTE256()
            pKeyBuffers_old = PBYTE256()

            SendMessage(hwnd, w.WM_ACTIVATE, w.WA_ACTIVE, 0)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, True)
            GetKeyboardState(ctypes.byref(pKeyBuffers_old))

            for modkey in shift:
                if modkey == w.VK_MENU:
                    lparam = lparam | 0x20000000
                    msg_down = w.WM_SYSKEYDOWN
                    msg_up = w.WM_SYSKEYUP
                pKeyBuffers[modkey] |= 128

            SetKeyboardState(ctypes.byref(pKeyBuffers))
            time.sleep(0.01)
            PostMessage(hwnd, msg_down, key, lparam)
            time.sleep(0.01)
            PostMessage(hwnd, msg_up, key, lparam | 0xC0000000)
            time.sleep(0.01)
            SetKeyboardState(ctypes.byref(pKeyBuffers_old))
            time.sleep(0.01)
            AttachThreadInput(GetCurrentThreadId(), ThreadId, False)

        else:
            SendMessage(hwnd, msg_down, key, lparam)
            SendMessage(hwnd, msg_up, key, lparam | 0xC0000000)


# # 엔터
def SendReturn(hwnd):
    win32api.PostMessage(hwnd, win32con.WM_KEYDOWN, win32con.VK_RETURN, 0)
    time.sleep(0.01)
    win32api.PostMessage(hwnd, win32con.WM_KEYUP, win32con.VK_RETURN, 0)


# # 채팅방 열기
def open_chatroom(chatroom_name):
    # # # 채팅방 목록 검색하는 Edit (채팅방이 열려있지 않아도 전송 가능하기 위하여)
    hwndkakao = win32gui.FindWindow(None, "카카오톡")
    hwndkakao_edit1 = win32gui.FindWindowEx( hwndkakao, None, "EVA_ChildWindow", None)
    hwndkakao_edit2_1 = win32gui.FindWindowEx( hwndkakao_edit1, None, "EVA_Window", None)
    hwndkakao_edit2_2 = win32gui.FindWindowEx( hwndkakao_edit1, hwndkakao_edit2_1, "EVA_Window", None)    # ㄴ시작핸들을 첫번째 자식 핸들(친구목록) 을 줌(hwndkakao_edit2_1)
    hwndkakao_edit3 = win32gui.FindWindowEx( hwndkakao_edit2_2, None, "Edit", None)

    # # Edit에 검색 _ 입력되어있는 텍스트가 있어도 덮어쓰기됨
    win32api.SendMessage(hwndkakao_edit3, win32con.WM_SETTEXT, 0, chatroom_name)
    time.sleep(1)   # 안정성 위해 필요
    SendReturn(hwndkakao_edit3)
    time.sleep(1)


# # 채팅내용 초기 저장 _ 마지막 채팅
def chat_last_save():
    open_chatroom(kakao_opentalk_name)  # 채팅방 열기
    ttext = copy_chatroom(kakao_opentalk_name)  # 채팅내용 가져오기

    a = ttext.split('\r\n')   # \r\n 으로 스플릿 __ 대화내용 인용의 경우 \r 때문에 해당안됨
    df = pd.DataFrame(a)    # DF 으로 바꾸기

    df[0] = df[0].str.replace('\[([\S\s]+)\] \[(오전|오후)([0-9:\s]+)\] ', '')  # 정규식으로 채팅내용만 남기기

    return df.index[-2], df.iloc[-2, 0]

# # 채팅방 커멘드 체크
def chat_chek_command(cls, clst):
    open_chatroom(kakao_opentalk_name)  # 채팅방 열기
    ttext = copy_chatroom(kakao_opentalk_name)  # 채팅내용 가져오기

    a = ttext.split('\r\n')  # \r\n 으로 스플릿 __ 대화내용 인용의 경우 \r 때문에 해당안됨
    df = pd.DataFrame(a)  # DF 으로 바꾸기

    df[0] = df[0].str.replace('\[([\S\s]+)\] \[(오전|오후)([0-9:\s]+)\] ', '')  # 정규식으로 채팅내용만 남기기

    if df.iloc[-2, 0] == clst:
        print("채팅 없었음..")
        return df.index[-2], df.iloc[-2, 0]
    else:
        print("채팅 있었음")

        df1 = df.iloc[cls+1 : , 0]   # 최근 채팅내용만 남김

        found = df1[ df1.str.contains(chat_command) ]    # 챗 카운트

        if 1 <= int(found.count()):
            print("-------커멘드 확인!")
            p_time_ymd_hms = \
                f"{time.localtime().tm_year}-{time.localtime().tm_mon}-{time.localtime().tm_mday} / " \
                f"{time.localtime().tm_hour}:{time.localtime().tm_min}:{time.localtime().tm_sec}"
            realtimeList = naver_realtimeList()  # 네이버 실시간 검색어 상위 20개
            kakao_sendtext(kakao_opentalk_name, f"{p_time_ymd_hms}\n{realtimeList}")  # 메시지 전송, time/실검

            # 명령어 여러개 쓸경우 리턴값으로 각각 빼서 쓰면 될듯. 일단 테스트용으로 위에 하드코딩 해둠
            return df.index[-2], df.iloc[-2, 0]

        else:
            print("커멘드 미확인")
            return df.index[-2], df.iloc[-2, 0]


# # 네이버 실검 상위 20개, 리턴
def naver_realtimeList():
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

    url = 'https://datalab.naver.com/keyword/realtimeList.naver?where=main'
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.findAll('span','item_title')

    a = []
    for item in data:
        a.append(item.get_text())

    s = "\n".join(a)
    return s


# # 스케줄러 job_1
def job_1():
    p_time_ymd_hms = \
        f"{time.localtime().tm_year}-{time.localtime().tm_mon}-{time.localtime().tm_mday} / " \
        f"{time.localtime().tm_hour}:{time.localtime().tm_min}:{time.localtime().tm_sec}"

    open_chatroom(kakao_opentalk_name)  # 채팅방 열기
    realtimeList = naver_realtimeList()  # 네이버 실시간 검색어 상위 20개
    kakao_sendtext(kakao_opentalk_name, f"{p_time_ymd_hms}\n{realtimeList}")  # 메시지 전송, time/실검


def main():

    # sched = BackgroundScheduler()
    # sched.start()

    cls, clst = chat_last_save()  # 초기설정 _ 마지막채팅 저장

    # # 매 분 5초마다 job_1 실행
    # sched.add_job(job_1, 'cron', second='*/5', id="test_1")

    while True:
        print("실행중.................")
        cls, clst = chat_chek_command(cls, clst)  # 커멘드 체크
        time.sleep(5)


if __name__ == '__main__':
    # main()
    copy_chatroom("상준")

2023년 4월 20일 목요일
[이상현] [오후 8:54] 진혁아 안녕?
2023년 5월 12일 금요일
[이상현] [오후 10:46] 어찌 잘 지내고 있니?
[이상현] [오후 10:48] 어찌 잘 지내고 있니?
2023년 5월 13일 토요일
[이상현] [오전 4:25] 어찌 잘 지내고 있니?

